<a href="https://colab.research.google.com/github/Hatice-Kocabas/Milk-Quality-/blob/master/DNNmilk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [9]:
data=pd.read_csv('/content/milknew.csv')

In [10]:
data.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


In [11]:
# Changing 'grade' data value to numerical value
data['Grade']=data['Grade'].map({'low':0,'medium':1,'high':2})
data.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,2
1,6.6,36,0,1,0,1,253,2
2,8.5,70,1,1,1,1,246,0
3,9.5,34,1,1,0,1,255,0
4,6.6,37,0,0,0,0,255,1


In [12]:
# Splitting Data
X = data.drop('Grade', axis=1).values.astype(np.float32)
y = data['Grade'].values.astype(np.int64)

In [21]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.25, random_state=42)

In [22]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [23]:
X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_val_tensor = torch.tensor(X_val)
y_val_tensor = torch.tensor(y_val)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)


In [24]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=8)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=8)

In [26]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(7, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)  # Çünkü 3 sınıf var: low, medium, high

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [27]:
# Model, loss fonksiyonu ve optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
epochs = 50
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Her epoch sonrası Validation set üzerindeki loss'u hesaplayıp yazdırma
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    val_accuracy = correct / total
    print(f"Epoch {epoch+1}/{epochs} - Training Loss: {running_loss / len(train_loader)}, Validation Loss: {val_loss / len(val_loader)}, Validation Accuracy: {val_accuracy * 100:.2f}%")

Epoch 1/50 - Training Loss: 0.00215652769747976, Validation Loss: 0.12602900099631775, Validation Accuracy: 99.06%
Epoch 2/50 - Training Loss: 0.0019690602287425916, Validation Loss: 0.12699876046786407, Validation Accuracy: 99.06%
Epoch 3/50 - Training Loss: 0.0017947933038442443, Validation Loss: 0.1275932354235044, Validation Accuracy: 99.06%
Epoch 4/50 - Training Loss: 0.0016393943320508696, Validation Loss: 0.1283619596699484, Validation Accuracy: 99.06%
Epoch 5/50 - Training Loss: 0.0015088727006201453, Validation Loss: 0.12912510764006044, Validation Accuracy: 99.06%
Epoch 6/50 - Training Loss: 0.0013784007828775203, Validation Loss: 0.1298336223660078, Validation Accuracy: 99.06%
Epoch 7/50 - Training Loss: 0.001271374154583782, Validation Loss: 0.1307860331350336, Validation Accuracy: 99.06%
Epoch 8/50 - Training Loss: 0.0011698066645436712, Validation Loss: 0.1314731370625345, Validation Accuracy: 99.06%
Epoch 9/50 - Training Loss: 0.0010791984720256663, Validation Loss: 0.13

In [31]:
# Test
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Test verisi doğruluğu: {accuracy * 100:.2f}%')

Test verisi doğruluğu: 99.06%
